<a id="Top"></a>

# Finding the Rotation Curve of an Asteroid or Comet with TESScut and lightkurve: Solutions

Before the solutions can be run, this code reproduces the necessary variables from the tutorial into this notebook.

# Contents
* [Imports, prerequisite cells](#Imports,-Prerequisite-Cells)
1. [Confirm lightcurve truncation is appropriate.](#1)
2. [Confirm remove_outlier cutoff value is appropriate.](#2)
3. [Is rotation period changed by sector?](#3)
4. [Use the TESSCut website to download more observations.](#4)
5. [Find the rotation period of Hippodamia.](#5)
6. [Why is the median flux of first sector's observations lower than the second?](#6)


# Imports, Prerequisite Cells

In [ ]:
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from astroquery.mast import Tesscut
from astropy.visualization import time_support
import lightkurve as lk

%matplotlib inline

In [ ]:
objname='Eleonora'
nsectors=2
for i in range(nsectors):
    tpf = lk.TessTargetPixelFile(f"objname_{i}.fits")
    if i==0: 
        tpfc=lk.TargetPixelFileCollection([tpf])
    else:
        tpfc.append(tpf)

In [ ]:
for i in range(nsectors):
    lc=tpfc[i].to_lightcurve()
    if i==0: 
        lcc=lk.LightCurveCollection([lc])
    else:
        lcc.append(lc)    
lcc[0]=lcc[0].truncate(after=293,column='cadenceno')
lcc[1]=lcc[1].truncate(before=59,column='cadenceno')
lcc[1]=lcc[1].remove_outliers(sigma_upper=2)

<a id="1"></a>
# 1. Confirm that the first lightcurve in our hdulist should be truncated after cadence number 293. 

In [ ]:
# Place for code for Exercise 1
# The easiest way to do this is to inspect the first TPF and see that 
# cadence 293 is the last one before one column of the image becomes cut off.
tpfc[0].interact()

<a id="2"></a>
# 2. Confirm that using a `remove_outlier` cutoff of `sigma_upper=2` did not affect the other cadences of the lightcurve for the second sector. 

At this point in the tutorial, the second sector's original light curve is still saved under the variable name `lc`; plot a graph of its time vs. flux columns and then overplot our sigma-clipped time and flux columns. Hint: You'll need to load `time_support()` first in order to use the time column in matplotlib. 

In [ ]:
# SOLUTION FOR EXERCISE 2
time_support()
plt.plot(lc.time,lc.flux)
plt.plot(lcc[1].time,lcc[1].flux)
plt.ylim(30000,36000) # Need to zoom in to see the two.

<a id="3"></a>
# 3. Determine if the rotation period returned by `lightkurve` is different when using only one sector's light curve at a time. 

In [ ]:
# SOLUTION FOR EXERCISE 3
pg0=lcc[0].to_periodogram()
pg1=lcc[1].to_periodogram()
print(pg0.period_at_max_power.to(u.hr))
print(pg1.period_at_max_power.to(u.hr))
# To 3 significant figures, lightkurve returns the same period (in hours) 
# with either light curve individually or with the two light curves combined.

<a id="4"></a>
# 4. Use the interactive [TESSCut webpage](https://mast.stsci.edu/tesscut/) to download 10x10 cutouts for all sectors observing Eleonora. 
Be sure to select "Moving Target" at the top! Load the resulting fits files in and confirm that they are identical to the items in `hdulist` above. 

**Hint:** Astropy has a built-in method called [FITSDiff](https://docs.astropy.org/en/stable/io/fits/api/diff.html). 

In [ ]:
# SOLUTION FOR EXERCISE 4
# Using the default filenames from the website at the time of the writing of this tutorial:
diff1=fits.FITSDiff('Eleonora_0.fits','Eleonora_1468.2870040465345-1475.8494727152606_10.0-x-10.0_astrocut.fits',numdiffs=-1)
print(diff1.identical)
print(diff1.report())
# The only differences appera to be in "CHECKSUM" and "DATASUM" values, which seem to correspond to the 
# timestamps when the files were saved. Try ignoring those values. 
diff1_ignore=fits.FITSDiff('Eleonora_0.fits',
                           'Eleonora_1468.2870040465345-1475.8494727152606_10.0-x-10.0_astrocut.fits',
                           numdiffs=-1,ignore_keywords=['CHECKSUM','DATASUM'])
print(diff1_ignore.identical)
# Do the same for the other file.
diff2_ignore=fits.FITSDiff('Eleonora_1.fits',
                           'Eleonora_1945.4743041992188-1954.8493041992188_10.0-x-10.0_astrocut.fits',
                           numdiffs=-1,ignore_keywords=['CHECKSUM','DATASUM'])
print(diff2_ignore.identical)

<a id="5"></a>
# 5. Try recreating the procedure above to find the rotation period for a fainter (higher magnitude) small body from [Pál et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract) such as Hippodamia. 
What issues do you encounter in this case?

In [ ]:
# SOLUTION FOR EXERCISE 5
objname2='Hippodamia'
sector_table2=Tesscut.get_sectors(objectname=objname2, moving_target=True)
print(sector_table2)
# Because there is only one sector, we do not need the loops
# that we did for Eleonora. 
hdulist2 = Tesscut.get_cutouts(objectname=objname2, moving_target=True, size=10)

In [ ]:
i=0
hdulist2[i].writeto(objname2+'_'+str(i)+'.fits',overwrite=True)
tpf2 = lk.TessTargetPixelFile(objname2+'_'+str(i)+'.fits')
tpf2.interact()

After scrolling through the cadences, it's clear that Hippodamia passes in front of many stars that significantly impact the flux in the aperture, because it is so much dimmer by itself. The influence on the light curve is much more pronounced than for Eleonora. We can try to use a similar outlier removal, but so many of the flux data points are high, it will likely not work. Generally, differential photometry (comparing images before the asteroid appears and after) is required. Because Hippodamia's period is relatively short, (P=8.9993 hr), we can try to cutout the few days around cadences 365-460 to see if we can find the rotation period from that.

In [ ]:
lc2=tpf2.to_lightcurve()
lc2=lc2.truncate(before=365,after=460,column='cadenceno')
lc2.plot()

In [ ]:
pg2=lc2.to_periodogram()
pg2.plot()
print(pg2.period_at_max_power.to(u.hr))
print(2*pg2.period_at_max_power.to(u.hr))


In [ ]:
# Again, our result is close to 1/2 times to the accepted period. 
lc2.fold(period=2*pg2.period_at_max_power).scatter()

<a id="6"></a>
# 6. Use [astroquery's Minor Planet Center Queries (MPC)](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html?highlight=mpc#ephemerides) `get_ephemeris` feature to investigate why the median flux for the first sector's observations is lower than for the second sector's observations. 
The times of each cadence are in the light curve's `time` column.

In [ ]:
# SOLUTION FOR EXERCISE 6.
from astroquery.mpc import MPC
# Get tables for each start and end time. Just view the first row.
eph1=MPC.get_ephemeris('Eleonora',start=lcc[0].time[0])
eph2=MPC.get_ephemeris('Eleonora',start=lcc[0].time[-1])
eph3=MPC.get_ephemeris('Eleonora',start=lcc[1].time[0])
eph4=MPC.get_ephemeris('Eleonora',start=lcc[1].time[-1])
columns=['Date','RA','Dec','Delta','r','Elongation','Phase','V']
print(eph1[columns][0])
print(eph2[columns][0])
print(eph3[columns][0])
print(eph4[columns][0])

In [ ]:
# Compare the medians of the two fluxes.
print(np.median(lcc[0].flux))
print(np.median(lcc[1].flux))
print((np.median(lcc[1].flux)-np.median(lcc[0].flux))/np.median(lcc[0].flux))

During the second sector, Eleonora was 2.0-2.2% closer than the first, but the median flux was nearly 12% higher and the MPC doesn't report any difference in V magnitude.
***
[Back to top](#Top)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 